In [68]:
from iotbx.map_manager import map_manager as MapManager
from mmtbx.model import manager as ModelManager
from iotbx.data_manager import DataManager
from iotbx.map_model_manager import map_model_manager as MapModelManager

from libtbx import group_args
from mmtbx.maps.correlation import five_cc

## Define functions

In [115]:
def extract_ligands(model,desired_resnames = [], ignored_resnames=["UNK"]):
    """
    :param model: A mmtbx.model.model_manager object
    :param desired_resnames: a list of residue names to extract. If empty, extract all
    :param ignored_resnames: a list of residue names to ignore
    
    :return: a list of group_args objects the the properties:
        resname: the residue name
        ligand_model: a model object containing only one ligand
        
    """
    
    model = dm.get_model()

    # this is necessary for cryo-em models without crystal symmetry. Maybe move this upstream later
    if model.crystal_symmetry() is None:
        from cctbx.maptbx.box import shift_and_box_model
        model = shift_and_box_model(model)

    
    sel_str_other = "not (water or nucleotide or protein)"
    other_model = model.select(model.selection(sel_str_other))
    ligand_models = []
    for rg in other_model.get_hierarchy().residue_groups():
        unique_resnames = rg.unique_resnames()
        assert (len(unique_resnames)==1) # at the moment, anticipate that each residue group has 1 residue name
        resname = unique_resnames[0]
        if resname not in ignored_resnames:
            if (len(desired_resnames) ==0) or resname in desired_resnames:
                atoms = rg.atoms()
                atoms_sel = atoms.extract_i_seq()
                ligand_model = other_model.select(atoms_sel)
                group = group_args(resname=resname,model=ligand_model)
                ligand_models.append(group)
    return ligand_models

In [136]:
def ligands_ccmask(data_manager,d_min=1):
    """
    :param data_manager: A data manager instance that contains a model and a real map
    :param d_min: the resolution of the real space map contained in data manager. (maybe accessible via data_manager?)
    :return: 
    """
    dm = data_manager
    
    ligands = extract_ligands(model)
    
    ccmask_values = []
    for ligand in ligands:
        map_model_manager=MapModelManager(map_manager=dm.get_real_map(), model=ligand.model,ignore_symmetry_conflicts=True)
        boxed_mmm = map_model_manager.extract_all_maps_around_model()
        ligand_mm=boxed_mmm.map_manager()
        ligand_model=boxed_mmm.model()
        
        # five_cc is pretty slow... can we use a lower level func?
        five_cc_obj = five_cc(
                       ligand_mm.map_data(),
                       ligand_model.get_xray_structure(),
                       d_min,
                       box=None,
                       keep_map_calc=False,
                       compute_cc_box=False,
                       compute_cc_image=False,
                       compute_cc_mask=True,
                       compute_cc_volume=False,
                       compute_cc_peaks=False)
 
        

        ligand.ccmask = five_cc_obj.result()["cc_mask"]
    return ligands

## Instantiate a DataManager object from data on disk

In [90]:
model_path = "data/6f1u_4169/6f1u_4169.cif"
map_path = "data/6f1u_4169/6f1u_4169.map"


dm = DataManager()
dm.process_model_file(model_path)
dm.process_real_map_file(map_path)
model = dm.get_model()

# this is necessary for cryo-em models without crystal symmetry. Maybe move this upstream later
if model.crystal_symmetry() is None:
    from cctbx.maptbx.box import shift_and_box_model
    model = shift_and_box_model(model)

In [138]:
ligands_results = ligands_ccmask(dm,d_min=3.4)

In [139]:
ligands_results

[group_args
   ccmask                         : 0.841597012587
   model                          : Model manager 
 Unit cell: (804, 804, 804, 90, 90, 90) Space group: P 1 (No. 1)
 Chains: 1 Residues 1 
 Working coordinate shift (0.0, 0.0, 0.0))
   resname                        : ADP, group_args
   ccmask                         : 0.820953325418
   model                          : Model manager 
 Unit cell: (804, 804, 804, 90, 90, 90) Space group: P 1 (No. 1)
 Chains: 1 Residues 1 
 Working coordinate shift (0.0, 0.0, 0.0))
   resname                        : ADP, group_args
   ccmask                         : 0.810208459439
   model                          : Model manager 
 Unit cell: (804, 804, 804, 90, 90, 90) Space group: P 1 (No. 1)
 Chains: 1 Residues 1 
 Working coordinate shift (0.0, 0.0, 0.0))
   resname                        : ADP]